In [1]:
import os

os.chdir("../../../../..")
%pwd

'/Users/davidfitzek/git/qflow-opt-scripts/libs/qflow'

In [2]:
import pandas as pd
import yaml
from qflow.utils.path import Path

/Users/davidfitzek/miniforge3/envs/qflow/lib/python3.10/site-packages/beartype/_util/hint/pep/utilpeptest.py:345: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.List deprecated by PEP 585 scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". See this discussion for further details and alternatives:
    https://github.com/beartype/beartype#pep-585-deprecations
  warn(


### Load the csv file

In [3]:
num_layers = 2
path = Path(
    [
        "resources",
        "test_logs",
        "h4",
        "num_qubits_8",
        f"num_layers_{num_layers}",
        "use_hf_params_False",
        "h4_1",
    ]
)

In [4]:
with open(f"{path}/config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
print(config)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/davidfitzek/git/qflow-opt-scripts/libs/qflow/resources/test_logs/h4/num_qubits_8/num_layers_2/use_hf_params_False/h4_1/config.yaml'

In [ ]:
df = pd.read_csv(f"{path}/data.csv")
df.head()

,Unnamed: 0,step,exp_val,optimizer,average,num_circuit_evaluations,params
0,0,0,-1.817263,Ground state,0,0,[]
1,1,1,-1.817263,Ground state,0,100,[]
2,2,2,-1.817263,Ground state,0,200,[]
3,3,3,-1.817263,Ground state,0,300,[]
4,4,4,-1.817263,Ground state,0,400,[]


### Find min exp val

We group the df by optimizer and search for the minimum exp_val. We also store the params and optimizer name for each minimum value. 

In [ ]:
# group the dataframe by the "optimizer" column
df_grouped = df.groupby("optimizer")
# get the index of the rows with the minimum "value" for each group
min_index = df_grouped["exp_val"].idxmin()
# get the rows with the minimum "value" for each group
min_values = df.loc[min_index]

list_optimizer = []
list_params = []
list_exp_val = []
for i, row in min_values.iterrows():
    print(row["optimizer"], row["exp_val"])
    # print(row["params"])
    list_params.append(row["params"])
    list_optimizer.append(row["optimizer"])
    list_exp_val.append(row["exp_val"])

AdamOptimizer -0.4166362797510073
Ground state -1.8172626605878184
QBroydenAdamOptimizer -1.4134254283493422
QBroydenOptimizer -1.4038661684580582
QNG2Optimizer -0.4166362797510017


### Run the circuit with the correct params and verify result 

In [ ]:
from qflow.templates.examples import h4_vqe_circuit
import pennylane as qml
import numpy as np

/Users/davidfitzek/miniforge3/envs/qflow/lib/python3.10/site-packages/beartype/_util/hint/pep/utilpeptest.py:345: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Tuple[numpy.ndarray, numpy.ndarray] deprecated by PEP 585 scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". See this discussion for further details and alternatives:
    https://github.com/beartype/beartype#pep-585-deprecations
  warn(


In [ ]:
circuit = h4_vqe_circuit(num_layers=num_layers)
params = circuit.init()

converged SCF energy = -1.78094876341636


In [ ]:
# sanity check
dev = qml.device("default.qubit", wires=circuit.wires)


@qml.qnode(dev)
def fun(x):
    circuit(x)
    return qml.expval(circuit.H)


for params, optimizer, exp_val_ground_truth in zip(
    list_params, list_optimizer, list_exp_val
):
    params = params.replace("[", "").replace("]", "").split()
    params = [float(param) for param in params]
    if len(params) == 0:
        continue

    exp_val = fun(params)
    print(
        f"{optimizer}: {np.round(exp_val, 3)} from previous simulation: {np.round(exp_val_ground_truth, 3)}"
    )
    # check that exp_val is close to exp_val_ground_truth
    assert np.isclose(exp_val, exp_val_ground_truth, atol=1e-1)

AdamOptimizer: -0.417 from previous simulation: -0.417
QBroydenAdamOptimizer: -1.413 from previous simulation: -1.413
QBroydenOptimizer: -1.404 from previous simulation: -1.404
QNG2Optimizer: -0.417 from previous simulation: -0.417
